In [1]:
from bs4 import BeautifulSoup, SoupStrainer
import requests
import httplib2
import os
from os.path import exists
import csv
import copy

In [2]:
categories = []
categories_occurences = []
omittedcategories = [["sports", "cricket", "football", "tennis"], ["world", "gulf", "americas", "mena", "asia"]]
words = [" and ", " or ", " nor ", " if ", " because ", " unless "]
links = open("websites.txt" , "r")
website_list = links.read()
website_list = website_list.split("\n")

### Links extractor

In [7]:
for link in website_list:
    url_extract = requests.get(link).text
    s = ""
    if link == "https://nypost.com/":
        link = link + "20"
    soup = BeautifulSoup(url_extract, )
    texts = soup.find_all('a')
    for L in texts:
        s+= str(L)
    link_splitted = link.split('/')
    website_name = link_splitted[2]
    website_name = website_name.replace("www.","")
    website_name = website_name.replace(".com","")
    s = set(s.split('"'))
    path = '../Research project/'+website_name
    if not os.path.exists(path):
        os.makedirs(path)
    if not os.path.exists(path + '/texts'):
        os.makedirs(path+ '/texts')
    outFile = open(path+"/"+website_name+"_Links.txt", "w")
    for x in s:
        if x.startswith(link) and '-' in x and len(x) >60 and not (".svg" in x or ".png" in x):
            outFile.write(x + "\n")

### Artical text extractor

In [9]:
for link in website_list:
    inFile = open('../Research project/'+website_name+'/'+website_name+"_Links.txt", "r")
    links = inFile.readlines()
    for n in links:
        n = n.strip()
        outFile = open('../Research project/'+website_name+'/texts/'+n.split('/')[3]+".txt", "a+"  , encoding="utf-8")
        url_extract = requests.get(n).text
        bool = 'true'
        genre = n.split('/')[3]
        #and_count = n.count
        for category in categories:
            if category == genre:
                bool = 'false'
        if bool == 'true':
            categories.append(genre)
        s = ""
        soup = BeautifulSoup(url_extract, )
        texts = soup.find_all('p')
        for L in texts:
            s+=L.text
        s= s.split('.')
        outFile.write('\n')
        for x in s:
            outFile.write(x)
            outFile.write(".\n")
    outFile.close()

In [ ]:
for link in website_list:
    for lists in omittedcategories:
        for omit in range(1, len(lists)):
            if lists[omit] in categories:
                categories.remove(lists[omit])
            if os.path.exists("../Research project/khaleejtimes/texts/"+lists[omit]+".txt"):
                inputFile = open("../Research project/khaleejtimes/texts/"+lists[omit]+".txt", "r")
                outputFile = open("../Research project/khaleejtimes/texts/"+lists[0]+".txt", "a")
                text = inputFile.read()
                outputFile.write(text)
                os.remove("../Research project/khaleejtimes/texts/"+lists[omit]+".txt")

    for category in categories:
        inputFile = open("../Research project/khaleejtimes/texts/"+category+".txt", "r")
        TextExtracted = inputFile.read()
        occurences = []
        for word in words:
            occurences.append(TextExtracted.lower().count(word))
        categories_occurences.append(occurences)
        inputFile.close()
    total = [0,0,0,0,0,0]
    for occurence in categories_occurences:
        for i in range(len(occurence)):
            total[i] = total[i] + occurence[i]
    statistics = open(path + "/Statistics.txt", "w")
    CSVstatistics = open(path + "/Statistics.csv", "w")
    csvwriter = csv.writer(CSVstatistics)
    statistics.write("Total occurences:\n")
    for index in range(len(words)):
        statistics.write(words[index]+" = "+ str(total[index])+" occurences.\n")
    statistics.write("Statistics per category:\n")
    statistics.write('{:35s}'.format("Category:"))
    for word in words:
        statistics.write('|' + '{:5s}'.format(word))
    statistics.write("\n")
    for index in range(len(categories)):
        statistics.write('{:35s}'.format(categories[index]))
        for q in range(len(categories_occurences[0])):
            statistics.write('|' + '{:5s}'.format(str(categories_occurences[index][q])))
        statistics.write("\n")
    csvStat = copy.deepcopy(categories_occurences)
    for index in range(len(categories)):
        csvStat[index].append(categories[index])
        csvStat[index] = csvStat[index][-1:] + csvStat[index][:-1]
    csvwriter.writerows(csvStat)
    CSVstatistics.close()
    statistics.close()